In [1]:

from lean_interact import TempRequireProject, LeanREPLConfig, LeanServer, Command

config = LeanREPLConfig(verbose=True, project=TempRequireProject("mathlib"))

server = LeanServer(config)


Build completed successfully.


In [2]:

server.run(Command(cmd="""import Mathlib
theorem ex_mathlib (x : ℝ) :
  x+0=x := by
    have h (x : ℝ) : x+0=x:= by simp
  exact h""", root_goals=True, ))

ValidationError: 1 validation error for CommandResponse
infotree
  Input should be a valid dictionary [type=dict_type, input_value=[{'node': {'stx': {'range...dInfo', 'children': []}], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/dict_type